# Задание 5. Применение дискретного ряда Фурье для решения методом сеток однородного уравнения теплопроводности
## Грунина Маргарита. Вариант 5

$\displaystyle \varphi(x) = \sin(\frac{\pi}{3} x (x - 1))$

In [1]:
import numpy as np
import pandas as pd
from scipy import integrate

In [3]:
# функция варианта задания
def phi(x):
    return np.sin(np.pi * x * (x - 1) / 3)

# вид ортонормированной собственной функции
def psi(x, p):
    return np.sqrt(2) * np.sin(np.pi * x * p)

T = 0.1

### 1) Аналитическое решение через ряд Фурье

In [15]:
P = 15

def coefs_Fou(P):
    coefs = []
    for p in range(1, P):
        coefs.append(integrate.quad(lambda x: phi(x) * psi(x, p), 0, 1)[0])
    return coefs

def Fou_solve(coefs, x, t):
    res = 0
    for p in range(len(coefs)):
        res += coefs[p] * np.exp(-np.pi ** 2 * (p + 1) ** 2 * t) * psi(x, p + 1)
    return res

x_arr = np.linspace(0, 1, 6)
t_arr = np.linspace(0, T, 6)

uf = [] # сюда записываем аналитическое решение
for t in t_arr:
    arr = []
    for x in x_arr:
        if t == 0:
            arr.append(phi(x))
        else:
            arr.append(Fou_solve(coefs_Fou(P), x, t))
    uf.append(arr)

uf_table = pd.DataFrame(data = uf, index=t_arr, columns=x_arr)
uf_table.columns.name = "t \ x"
uf_table

t \ x,0.0,0.2,0.4,0.6,0.8,1.0
0.00,-0.0,-0.166769,-0.248690,-0.248690,-0.166769,0.000000e+00
0.02,0.0,-0.130936,-0.208022,-0.208022,-0.130936,-2.759562e-17
0.04,0.0,-0.106363,-0.171450,-0.171450,-0.106363,-2.221244e-17
0.06,0.0,-0.087121,-0.140855,-0.140855,-0.087121,-1.816035e-17
0.08,0.0,-0.071483,-0.115643,-0.115643,-0.071483,-1.489489e-17
0.10,0.0,-0.058673,-0.094931,-0.094931,-0.058673,-1.222466e-17


### 2) Аналитическое решение через дискретный ряд Фурье



In [11]:
N = 20

# Раскладываем в ДРФ начальную функцию
def coefs_discrete_Fou(N):
    h = 1 / N
    coefs = []
    for p in range(1, N):
        coef = 0
        for i in range(1, N):
            coef += phi(i * h) * psi(i * h, p)
        coef *= h
        coefs.append(coef)
    return coefs

# Строим решение
def discrete_Fou_solve(coefs, x, t):
    res = 0
    for p in range(len(coefs)):
        res += coefs[p] * np.exp(-np.pi ** 2 * (p + 1) ** 2 * t) * psi(x, p + 1)
    return res

udsf = [] # аналитическое решение через ДПФ
error = []
for i in range(len(t_arr)):
    arr_udsf = []
    arr_error = []
    for j in range(len(x_arr)):
        temp = discrete_Fou_solve(coefs_discrete_Fou(N), x_arr[j], t_arr[i])
        arr_udsf.append(temp)
        arr_error.append(abs(temp - Fou_solve(coefs_Fou(15), x_arr[j], t_arr[i])))
    udsf.append(arr_udsf)
    error.append(arr_error)


print("N = ", N)
print("||uf - udsf^(N)|| = max|uf - udsf^(N)| = ", max(max(error)))

N =  20
||uf - udsf^(N)|| = max|uf - udsf^(N)| =  4.861687182725061e-05


### 3) Сеточное решение: используем ДРФ по схеме с весами при N = 5, 10, 20 для различных значений параметра σ

In [12]:
def lamb(N, M, sigma):
    h = 1 / N
    tau = 0.1 / M
    l = []
    for p in range(1, N):
        tmp1 = 1 - (4 * (1 - sigma) * tau / h ** 2) * np.sin(p * np.pi * h / 2) ** 2
        tmp2 = 1 + (4 * sigma * tau / h ** 2) * np.sin(p * np.pi * h / 2) ** 2
        l.append(tmp1 / tmp2)
    return l

def grid_Fou_solve(N, M, coefs, l, x, t):
    h = 1 / N
    tau = 0.1 / M
    res = 0
    k = t / tau
    for p in range(len(coefs)):
        res += coefs[p] * (l[p] ** k) * psi(x, p + 1)
    return res

def max_error(N, M, sigma, x_arr, t_arr):
    u_grid = []
    error = []
    for i in range(len(t_arr)):
        arr_u_grid = []
        arr_error = []
        for j in range(len(x_arr)):
            temp = grid_Fou_solve(N, M, coefs_discrete_Fou(N), lamb(N, M, sigma), x_arr[j], t_arr[i])
            arr_u_grid.append(temp)
            if t_arr[i] == 0:
                arr_error.append(abs(temp - phi(x_arr[j])))
            else:
                arr_error.append(abs(temp - Fou_solve(coefs_Fou(10), x_arr[j], t_arr[i])))
        u_grid.append(arr_u_grid)
        error.append(arr_error)

    return max(max(error))

### 4) Таблица значений модуля разности uf и приближенных сеточных решений ||uf − u*||

In [14]:
N_M = [[5, 5], [10, 20], [20, 80], [20, 20]]
h_tau = [[0.2, 0.02], [0.1, 0.005], [0.05, 0.00125], [0.05, 0.005]]

def sigma(h_tau_index):
    return np.array([0, 1, 0.5, 0.5 -1 * h_tau[h_tau_index][0] ** 2 / (12 * h_tau[h_tau_index][1])])

x_arr = np.linspace(0, 1, 6)
t_arr = np.linspace(0, T, 6)

result = []
for i in range(len(N_M)):
    sigmas = sigma(i)
    arr_result = []
    for j in range(len(sigmas)):
        arr_result.append(max_error(N_M[i][0], N_M[i][1], sigmas[j], x_arr, t_arr))
    result.append(arr_result)

result = np.array(result).transpose()

error_grid_table = pd.DataFrame(data = result, index = ['σ = 0', 'σ = 1', 'σ = 1/2', 'σ = 1/2 - h ** 2 / (12 * τ)'], columns = ['(0.2, 0.02)', '(0.1, 0.005)', '(0.05, 0.00125)', '(0.05, 0.005)'])
error_grid_table.columns.name = "(h, τ)"
error_grid_table


"(h, τ)","(0.2, 0.02)","(0.1, 0.005)","(0.05, 0.00125)","(0.05, 0.005)"
σ = 0,0.006591,0.001105,0.000391,1.117629e+12
σ = 1,0.011203,0.003008,0.000765,2.449182e-03
σ = 1/2,0.002738,0.000748,0.000191,1.734852e-04
σ = 1/2 - h ** 2 / (12 * τ),0.001032,0.000055,0.000003,4.863421e-05
